In [98]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, models, transforms
torch.manual_seed(360)

import os
from urllib import request
from zipfile import ZipFile

In [94]:
import numpy as np
from sklearn.datasets import make_classification # Generate a random n-class classification problem.
from sklearn.datasets import make_regression # Generate a random regression problem.
from sklearn.pipeline import Pipeline # Sequentially apply a list of transforms and a final estimator.
from sklearn.preprocessing import StandardScaler # Standardize features by removing the mean and scaling to unit variance
from sklearn.model_selection import GridSearchCV # Exhaustive search over specified parameter values for an estimator.
from skorch import NeuralNetClassifier
from skorch import NeuralNetRegressor
from skorch.callbacks import EpochScoring
from skorch.helper import predefined_split
import pickle # Create portable serialized representations of Python objects.

### Classification

In [5]:
X, y = make_classification(n_samples = 1000, n_features = 20, n_informative = 10, random_state = 0)
X, y = X.astype(np.float32), y.astype(np.int64)

In [6]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

In [17]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X
    
model = NeuralNetClassifier(
    ClassifierModule,
    max_epochs = 20,
    lr = 0.1,
    device = 'cuda'
)

In [18]:
model.fit(X, y)
# Predictions
# y_pred = net.predict(X[:5])
# y_proba = net.predict_proba(X[:5])

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6794       0.6150        0.6741  0.8387
      2        0.6657       0.6350        0.6617  0.0396
      3        0.6520       0.6650        0.6503  0.0422
      4        0.6450       0.7050        0.6374  0.0402
      5        0.6358       0.7050        0.6248  0.0422
      6        0.6176       0.7000        0.6113  0.0408
      7        0.6088       0.7050        0.5990  0.0408
      8        0.5903       0.7000        0.5883  0.0411
      9        0.5922       0.7000        0.5775  0.0451
     10        0.5722       0.7150        0.5632  0.0409
     11        0.5711       0.7250        0.5553  0.0410
     12        0.5673       0.7250        0.5489  0.0399
     13        0.5521       0.7250        0.5482  0.0403
     14        0.5526       0.7300        0.5449  0.0411
     15        0.5446       0.7300        0.5364  0.0398
     16        0.5404       0.7

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

### Regression

In [22]:
X_regr, y_regr = make_regression(1000, 20, n_informative = 10, random_state = 0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32) / 100
y_regr = y_regr.reshape(-1, 1)

In [26]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

In [28]:
net_regr = NeuralNetRegressor(
    RegressorModule,
    max_epochs = 20,
    lr = 0.1,
    device = 'cuda'
)

In [29]:
net_regr.fit(X_regr, y_regr)
# Prediction
# y_pred = net_regr.predict(X_regr[:5])

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.6894        3.9830  0.1708
      2        4.5409        3.6905  0.0453
      3        3.6147        1.5435  0.0625
      4        0.8586        0.2304  0.0378
      5        0.2164        0.2580  0.0312
      6        0.3034        0.0829  0.0534
      7        0.1035        0.2024  0.0312
      8        0.2322        0.0599  0.0534
      9        0.0860        0.2011  0.0469
     10        0.1602        0.0426  0.0534
     11        0.0572        0.0939  0.0469
     12        0.0755        0.0410  0.0377
     13        0.0625        0.1398  0.0312
     14        0.0825        0.0281  0.0534
     15        0.0339        0.0414  0.0469
     16        0.0269        0.0224  0.0534
     17        0.0286        0.0470  0.0469
     18        0.0306        0.0240  0.0534
     19        0.0278        0.0357  0.0469
     20        0.0218        0.0190  0.0534


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=1, bias=True)
  ),
)

In [ ]:
# Saving the whole model
# file_name = '/tmp/mymodel.pkl'
# with open(file_name, 'wb') as f:
#     pickle.dump(net, f)
    
# with open(file_name, 'rb') as f:
#     new_net = pickle.load(f)

# Saving only the model parameters
# net.save_params(f_params = file_name)
# new_net = NeuralNetClassifier(
#     ClassifierModule,
#     max_epochs = 20,
#     lr = 0.1
# ).initialize()
# new_net.load_params(file_name)

### Pipeline

In [36]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', model),
])
pipe.fit(X, y)
# y_proba = pipe.predict_proba(X[:5])

Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6918       0.4900        0.6899  0.0846
      2        0.6925       0.5100        0.6885  0.0534
      3        0.6879       0.5800        0.6871  0.0312
      4        0.6888       0.6350        0.6857  0.0312
      5        0.6861       0.6500        0.6840  0.0534
      6        0.6843       0.6500        0.6819  0.0312
      7        0.6790       0.6550        0.6796  0.0534
      8        0.6799       0.6550        0.6769  0.0312
      9        0.6732       0.6650        0.6735  0.0534
     10        0.6727       0.6700        0.6699  0.0469
     11        0.6672       0.6850        0.6657  0.0469
     12        0.6613       0.6900        0.6605  0.0378
     13        0.6591       0.7050        0.6551  0.0499
     14        0.6511       0.7000        0.6487  0.0348
     15        0.6483       0.7100   

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('net',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
))],
         verbose=False)

### Callbacks

There is a scoring callback in skorch, EpochScoring, which we use for this. We have to specify which score to calculate. We have 3 choices:

* Passing a string: This should be a valid sklearn metric. For a list of all existing scores, look:
    https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics

* Passing None: If you implement your own .score method on your neural net, passing scoring=None will tell skorch to use that.

* Passing a function or callable: If we want to define our own scoring function, we pass a function with the signature func(model, X, y) -> score, which is then used.

Note that this works exactly the same as scoring in sklearn does.

In [38]:
auc = EpochScoring(scoring='roc_auc', lower_is_better=False)
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    callbacks=[auc],
)
net.fit(X, y)

  epoch    roc_auc    train_loss    valid_acc    valid_loss     dur
-------  ---------  ------------  -----------  ------------  ------
      1     0.6039        0.6872       0.5750        0.6822  0.0846
      2     0.6232        0.6824       0.5800        0.6732  0.0312
      3     0.6415        0.6700       0.6250        0.6651  0.0221
      4     0.6612        0.6698       0.6500        0.6584  0.0156
      5     0.6795        0.6544       0.6600        0.6498  0.0156
      6     0.6995        0.6479       0.6850        0.6378  0.0156
      7     0.7086        0.6353       0.6750        0.6282  0.0186
      8     0.7122        0.6329       0.6700        0.6198  0.0201
      9     0.7178        0.6135       0.6800        0.6085  0.0201
     10     0.7270        0.6053       0.7000        0.6006  0.0101
     11     0.7364        0.6050       0.7100        0.5904  0.0201
     12     0.7394        0.5821       0.7200        0.5824  0.0201
     13     0.7463        0.5790       0.7250   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

### GridSearchCV

In [42]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs = 20,
    lr = 0.1,
    verbose = 0,
    optimizer__momentum = 0.9,
)
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20],
    'module__dropout': [0, 0.5],
    'optimizer__nesterov': [False, True],
}
gs = GridSearchCV(net, params, refit = False, cv = 3, scoring = 'accuracy', verbose = 2)

In [43]:
gs.fit(X, y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False, total=   0.4s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False, total=   0.3s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False, total=   0.3s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True, total=   0.3s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True, total=   0.3s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True, total=   0.3s
[CV] lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False 
[CV]  lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False, total=   0.3s
[CV] lr=0.05, 

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:   14.0s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.ClassifierModule'>,
),
             iid='deprecated', n_jobs=None,
             param_grid={'lr': [0.05, 0.1], 'module__dropout': [0, 0.5],
                         'module__num_units': [10, 20],
                         'optimizer__nesterov': [False, True]},
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring='accuracy', verbose=2)

***

In [44]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X, y = X.astype(np.float32), y.astype(np.int64)

#### Writing a custom callback

* They should inherit from skorch.callbacks.Callback.
* They should implement at least one of the on_-methods provided by the parent class (e.g. on_batch_begin or on_epoch_end).
* As argument, the on_-methods first get the NeuralNet instance, and, where appropriate, the local data (e.g. the data from the current batch). The method should also have **kwargs in the signature for potentially unused arguments.
* Optional: If you have attributes that should be reset when the model is re-initialized, those attributes should be set in the initialize method.


In [45]:
from skorch.callbacks import Callback

In [48]:
def tweet(msg):
    print('~'*60)
    print('*tweet*', msg, '#skorch #pytorch')
    print('~'*60)
    
class AccuracyTweet(Callback):
    def __init__(self, min_accuracy):
        self.min_accuracy = min_accuracy
        
    def initialize(self):
        self.critical_epoch_ = -1
        
    def on_epoch_end(self, net, **kwargs):
        if self.critical_epoch_ > -1:
            return
        # look at the validation accuracy of the last epoch
        if net.history[-1, 'valid_acc'] >= self.min_accuracy:
            self.critical_epoch_ = len(net.history)
            
    def on_train_end(self, net, **kwargs):
        if self.critical_epoch_ < 0:
            msg = 'Accuracy never reached {} :('.format(self.min_accuracy)
        else:
            msg = 'Accuracy reached {} at epoch {}!!!'.format(
                    self.min_accuracy, self.critical_epoch_)
            
        tweet(msg)

In [49]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=15,
    lr=0.02,
    warm_start=True,
    callbacks=[AccuracyTweet(min_accuracy=0.7)],
)

In [50]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7041       0.4900        0.6958  0.0359
      2        0.7087       0.5100        0.6918  0.0199
      3        0.7028       0.5250        0.6896  0.0180
      4        0.6961       0.5550        0.6875  0.0180
      5        0.6960       0.5450        0.6853  0.0199
      6        0.6944       0.5700        0.6832  0.0239
      7        0.6895       0.5750        0.6813  0.0199
      8        0.6844       0.5850        0.6795  0.0190
      9        0.6871       0.5900        0.6777  0.0239
     10        0.6828       0.5900        0.6765  0.0180
     11        0.6816       0.5850        0.6751  0.0180
     12        0.6825       0.5900        0.6740  0.0189
     13        0.6772       0.5950        0.6728  0.0199
     14        0.6759       0.6050        0.6714  0.0199
     15        0.6773       0.6000        0.6701  0.0190
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

In [53]:
net.fit(X, y)

     46        0.6314       0.6900        0.6289  0.0229
     47        0.6255       0.6950        0.6275  0.0180
     48        0.6305       0.7000        0.6265  0.0180
     49        0.6204       0.7000        0.6247  0.0170
     50        0.6187       0.7000        0.6234  0.0180
     51        0.6208       0.6900        0.6220  0.0209
     52        0.6250       0.6900        0.6211  0.0180
     53        0.6069       0.6900        0.6197  0.0180
     54        0.6046       0.6950        0.6185  0.0170
     55        0.6054       0.7050        0.6170  0.0180
     56        0.6142       0.7100        0.6152  0.0170
     57        0.6109       0.7050        0.6142  0.0259
     58        0.6098       0.7000        0.6130  0.0170
     59        0.5872       0.6950        0.6112  0.0189
     60        0.5861       0.7050        0.6101  0.0170
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*tweet* Accuracy reached 0.7 at epoch 48!!! #skorch #pytorch
~~~~~~~~~~~~~~~~~~~~~~~

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

#### Working with Dataset

In [64]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
        assert len(X) == len(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [68]:
X, y = make_classification(1000, 20, n_informative = 10, random_state = 0)
X = X.astype(np.float32)
y = y.astype(np.int64)
dataset = MyDataset(X, y)

In [66]:
net = NeuralNetClassifier(ClassifierModule)

In [59]:
try:
    net.fit(dataset, y = None)
except ValueError as e:
    print("Error:", e)

Error: Stratified CV requires explicitely passing a suitable y.


In [70]:
net.fit(dataset, y = y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7107       0.5000        0.7123  0.0234
      2        0.7057       0.5000        0.7083  0.0095
      3        0.7034       0.5000        0.7043  0.0109
      4        0.6996       0.5050        0.7011  0.0189
      5        0.6924       0.5050        0.6983  0.0163
      6        0.6936       0.5150        0.6955  0.0196
      7        0.6870       0.5150        0.6932  0.0179
      8        0.6895       0.5250        0.6909  0.0199
      9        0.6840       0.5350        0.6888  0.0189
     10        0.6779       0.5400        0.6867  0.0167


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

In [86]:
X, y = make_classification(1000, 20, n_informative = 10, random_state = 0)
X = X.astype(np.float32)
y = y.astype(np.int64)
X0, X1 = X[:, :10], X[:, 10:]
X_dict = {'X0': X0, 'X1': X1}

In [81]:
#  When skorch passes the dict to the pytorch module, it will pass the data as keyword arguments to the 
#  forward call. That means that we should accept the two keys XO and X1 in the forward method, as shown below:

class ClassifierWithDict(nn.Module):
    def __init__(
            self,
            num_units0=50,
            num_units1=50,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierWithDict, self).__init__()
        self.num_units0 = num_units0
        self.num_units1 = num_units1
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(10, num_units0)
        self.dense1 = nn.Linear(10, num_units1)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.output = nn.Linear(num_units0 + num_units1, 2)

    def forward(self, X0, X1, **kwargs):
        X0 = self.nonlin(self.dense0(X0))
        X0 = self.dropout(X0)

        X1 = self.nonlin(self.dense1(X1))
        X1 = self.dropout(X1)

        X = torch.cat((X0, X1), dim=1)
        X = F.relu(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

In [84]:
net = NeuralNetClassifier(ClassifierWithDict, verbose=0)

In [87]:
net.fit(X_dict, y)

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierWithDict(
    (dense0): Linear(in_features=10, out_features=50, bias=True)
    (dense1): Linear(in_features=10, out_features=50, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (output): Linear(in_features=100, out_features=2, bias=True)
  ),
)

### Multiple return values from forward

In [90]:
from skorch import NeuralNetRegressor

class Encoder(nn.Module):
    def __init__(self, num_units = 5):
        super().__init__()
        self.num_units = num_units
        
        self.encode = nn.Sequential(
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, self.num_units),
            nn.ReLU()
        )
        
    def forward(self, X):
        encoded = self.encode(X)
        return encoded
    
class Decoder(nn.Module):
    def __init__(self, num_units):
        super().__init__()
        self.num_units = num_units
        
        self.decode = nn.Sequential(
            nn.Linear(self.num_units, 10),
            nn.ReLU(),
            nn.Linear(10, 20)
        )
        
    def forward(self, X):
        decoded = self.decode(X)
        return decoded
    
class AutoEncoder(nn.Module):
    def __init__(self, num_units):
        super().__init__()
        self.num_units = num_units

        self.encoder = Encoder(num_units=self.num_units)
        self.decoder = Decoder(num_units=self.num_units)
        
    def forward(self, X):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return decoded, encoded  # <- return a tuple of two values

In [91]:
class AutoEncoderNet(NeuralNetRegressor):
    def get_loss(self, y_pred, y_true, *args, **kwargs):
        decoded, encoded = y_pred
        loss_reconstruction = super().get_loss(decoded, y_true, *args, **kwargs)
        loss_l1 = 1e-3 * torch.abs(encoded).sum()
        return loss_reconstruction + loss_l1

In [92]:
net = AutoEncoderNet(
    AutoEncoder,
    module__num_units = 5,
    lr = 0.3,
)

In [93]:
net.fit(X, X)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        3.8231        3.7714  0.0808
      2        3.6923        3.6972  0.0300
      3        3.6327        3.6555  0.0257
      4        3.6000        3.6318  0.0250
      5        3.5821        3.6186  0.0250
      6        3.5724        3.6112  0.0259
      7        3.5673        3.6069  0.0268
      8        3.5641        3.6037  0.0244
      9        3.5617        3.6007  0.0250
     10        3.5589        3.5968  0.0259


<class '__main__.AutoEncoderNet'>[initialized](
  module_=AutoEncoder(
    (encoder): Encoder(
      (encode): Sequential(
        (0): Linear(in_features=20, out_features=10, bias=True)
        (1): ReLU()
        (2): Linear(in_features=10, out_features=5, bias=True)
        (3): ReLU()
      )
    )
    (decoder): Decoder(
      (decode): Sequential(
        (0): Linear(in_features=5, out_features=10, bias=True)
        (1): ReLU()
        (2): Linear(in_features=10, out_features=20, bias=True)
      )
    )
  ),
)

### Transfer Learning

In [100]:
def download_and_extract_data(dataset_dir = 'data'):
    data_zip = os.path.join(dataset_dir, 'himenoptera_data.zip')
    data_path = os.path.join(dataset_dir, 'hymenoptera_data')
    url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
    
    if not os.path.exists(data_path):
        if not os.path.exists(data_zip):
            print("Starting to download data...")
            data = request.urlopen(url, timeout = 15).read()
            with open(data_zip, 'wb') as f:
                f.write(data)
                
        print("Starting to extract data...")
        with ZipFile(data_zip, 'r') as zip_f:
            zip_f.extractall(dataset_dir)
            
    print("Data has been downloaded and extracted to {}.".format(dataset_dir))
    
download_and_extract_data()

Starting to download data...
Starting to extract data...
Data has been downloaded and extracted to data.


In [102]:
data_dir = 'data/hymenoptera_data'
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_ds = datasets.ImageFolder(
    os.path.join(data_dir, 'train'), train_transforms)
val_ds = datasets.ImageFolder(
os.path.join(data_dir, 'val'), val_transforms)

In [103]:
class PretrainedModel(nn.Module):
    def __init__(self, output_features):
        super().__init__()
        model = models.resnet18(pretrained = True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, output_features)
        self.model = model
        
    def forward(self, x):
        return self.model(x)